# Time Correlation Filter Function

The Purpose of this Notebook will be to to plan and test a time correlation function that will filter sources depending on how well its opitcal and radio data lines up timewise.

The goal so far is to collect the optical and radio data for all the eta-v filtered sources (206 of them), and put them into two dataframes: fsd for FINK data and vsd for VAST data.

I've managed to construct vsd fine, but I'm having issues constructing fsd. when I run the portal request for the full list of sources, even with batching, the kernel appears to die and the notebook is reset. This happens even when the request is contained in a function I've called 'query_fink_db', which you can find in Projecttools.py. This Kernel restart error dosent happen if the ID list is sufficiently small (I tested 12 IDS with batching and it worked fine.)

In [1]:
#here are the necessary imports
import os
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from io import StringIO
from vasttools.pipeline import Pipeline
from vasttools.query import Query
import Projecttools as pro #brand new module for frequently used code!

%matplotlib inline

In [2]:
cms = pd.read_pickle('Fink_2020_sources_matched_to_VAST_all_sources.pickle')
pro.family_sort(cms)
cms.groupby('family').size().sort_values(ascending=False)

family
AGN                827
Unknown            516
Galaxy             167
Solar System        81
Radio               70
Supernova           51
Multiwavelength     39
Star                21
dtype: int64

In [3]:
#This will automatically find the base directory that needed to be specified
pipe=Pipeline()
#this way, we can also load specific runs from the VAST pipeline:
my_run=pipe.load_run('tiles_corrected')

/opt/conda/lib/python3.9/site-packages/vasttools/pipeline.py:2609: UserWarning: Measurements have been loaded with vaex.
  warnings.warn("Measurements have been loaded with vaex.")


In [4]:
#Im just putting the eta and v threshholds because the eta-v analysis takes an actual eternity to complete and I already
#have the values here:
eta_thresh=2.315552652171963
v_thresh=0.2878888414273631

In [5]:
cms_candidates = pro.eta_v_candidate_filter(cms,my_run,eta_thresh,v_thresh)
cms_candidates.groupby('family').size().sort_values(ascending=False)

There are 213 candidate sources:


family
AGN                93
Unknown            53
Solar System       30
Galaxy             15
Radio               9
Star                5
Multiwavelength     4
Supernova           4
dtype: int64

I will be testing out this function on the ETA-V filtered sources. In order for this to work, I need to have both the radio and optical data available for each source. Since the FINK broker has a limit as to how many sources can be queried at a time, I will have to either take a random sample of sources from the filtered catalogue or apply the function to a sampling of each family seperately.

Alternatively, I can do some "Batching:" breaking up the ID list into batches, running the portal query, and stitching the results of each batch together into a DataFrame.

In [6]:
#These IDs are selected from the curated list of interesting sources (lightcurves can be seen via powerpoint.)
Idlist=cms_candidates['objectId']

In [ ]:
#defining column array for cutouts
cutouts=[
'b:cutoutScience_stampData',
'b:cutoutTemplate_stampData',
'b:cutoutDifference_stampData'
]

num_elem=len(Idlist)#length of id list
num_chunks=num_elem//10+1 #number of chunks, based on how you want to divide them up. in this case, 10 IDS per chunk
list_chunks=np.array_split(np.arange(num_elem), num_chunks) #np.arange(num_elem) makes an ordered array, from 0 to (num_elem - 1).
                                                            #np.array_split splits said ordered array according to the number of chunks specified by num_chunks
                                                            #each chunk is an element in the array 'list_chunks'

list_df=[] #empty array to hold fink sources.

for chunk_idx in list_chunks: #for each chunk in list_chunks
    start,end=chunk_idx[0],chunk_idx[-1]+1 #define the starting and ending indexes for the given chunk

#this is the request made to the fink portal to pull out the info for each source
    r = requests.post(
      'https://fink-portal.org/api/v1/objects',
      json={
        'objectId': ','.join(Idlist[start:end]), #This is where the 'chunk_idx[-1] +1' comes into play. the 'end' variable when slicing the list is inclusive of the index.
        'output-format': 'json',
        'withcutouts': 'True',
        'cols': ','.join(cutouts),
        'withupperlim': 'True' #important for lightcurve plotting
      }
    )
    df_tmp=pd.read_json(StringIO(r.content.decode()))#define a temporary dataframe that holds the queried sources from the chunk
    #df_tmp=pro.query_fink_db(Idlist[start:end])
    list_df.append(df_tmp)

fsd=pd.concat(list_df)

In [ ]:
fsd

In [ ]:
#cms_candidates_selection = cms_candidates.query('objectId == @Special_IDS_1')

#This reformats the index's in the candidate selection, so specific row ranges can be pulled out.
#cms_candidates_selection = cms_candidates_selection.reset_index()
#len(cms_candidates_selection)

In [ ]:
#This selects rows in our selection in interger index steps from a starting and stopping point
#test.loc[start:stop:steps]

#This selects up to 30 rows at a time
#candidate_sample = cms_candidates_selection.loc[0:11:1]
#len(candidate_sample)

In [ ]:
#defining column array for cutouts
#cutouts=[
#'b:cutoutScience_stampData',
#'b:cutoutTemplate_stampData',
#'b:cutoutDifference_stampData'
#]

#this is the request made to the fink portal to pull out the info for each source
#r = requests.post(
#'https://fink-portal.org/api/v1/objects',
# json={
#    'objectId': ','.join(Idlist), 
#    'output-format': 'json',
#    'withcutouts': 'True',
#    'cols': ','.join(cutouts),
#    'withupperlim': 'True' #important for lightcurve plotting
#  }
#)

In [ ]:
#reads in json file data as DataFrame. fsd stands for 'FINK source data'
#fsd=pd.read_json(StringIO(r.content.decode()))

In [ ]:
#fsd object should contain the optical time data for the selected objects
#fsd

In [ ]:
#vsd stands for 'vast source data'
#vsd=[]
#for x in Special_IDS_1:
    #y=cms_candidates_selection[cms_candidates_selection['objectId'] == x]['matched_id'].astype(int).values[0]
    #z=my_run.get_source(y).measurements
    #vsd.append(z)

As you can see below, the radio data is there.

In [8]:
vsi=[]
for i in Idlist:
    y=cms_candidates[cms_candidates['objectId'] == i]['matched_id'].astype(int).values[0]
    vsi.append(y)
meas=my_run.measurements
vsd=meas[meas.source.isin(vsi)]

In [16]:
#that should have a length of 206, because, we're neglecting duplicate lines
vsd

#,source,island_id,component_id,local_rms,ra,ra_err,dec,dec_err,flux_peak,flux_peak_err,flux_int,flux_int_err,bmaj,err_bmaj,bmin,err_bmin,pa,err_pa,psf_bmaj,psf_bmin,psf_pa,flag_c4,chi_squared_fit,spectral_index,spectral_index_from_TT,has_siblings,image_id,time,name,snr,compactness,ew_sys_err,ns_sys_err,error_radius,uncertainty_ew,uncertainty_ns,weight_ew,weight_ns,forced,flux_int_isl_ratio,flux_peak_isl_ratio,id
0,3363670,CS_0918-06A_island_3703,CS_0918-06A_component_3703a,0.28600001335144043,138.08865356445312,0.00057204783661291,-3.4263510704040527,0.0004306031623855233,1.5360000133514404,0.3036591708660126,2.1059999465942383,0.6530292630195618,25.65999984741211,0.0013834730489179492,17.229999542236328,0.0009637327748350799,108.5199966430664,0.3482147455215454,12.729999542236328,9.180000305175781,-84.70999908447266,False,0.3540000021457672,-0.2199999988079071,True,False,1133,2019-04-29 02:58:18.044000+00:00,SB8592_CS_0918-06A_component_3703a,5.37062931060791,1.37109375,0.00027777778450399637,0.00027777778450399637,0.0007151851314119995,0.0007672354695387185,0.0007672354695387185,1698801.5,1698801.5,False,1.0,1.0,16300152
1,3363670,SB10908_island_4202,SB10908_component_4202a,0.2939999997615814,138.08843994140625,0.0003439071006141603,-3.4262819290161133,0.00030216891900636256,1.5809999704360962,0.3116055727005005,2.1389999389648438,0.6638715267181396,14.890000343322754,0.0008290532277897,12.800000190734863,0.0006890709628351033,112.63999938964844,0.9026240706443787,8.670000076293945,5.190000057220459,-50.880001068115234,False,0.8859999775886536,-3.430000066757202,True,False,1449,2019-12-19 18:59:28.454000+00:00,SB10908_SB10908_component_4202a,5.377551078796387,1.3529411554336548,0.00027777778450399637,0.00027777778450399637,0.00045733587467111647,0.0005350855644792318,0.0005350855644792318,3492637.25,3492637.25,False,1.0,1.0,17974207
2,3363670,15670_island_4273,15670_component_4273a,0.30000001192092896,138.08871459960938,0.000240542518440634,-3.4262449741363525,0.00023910308664198965,1.5700000524520874,0.293329119682312,1.4450000524520874,0.4811488091945648,11.289999961853027,0.0005757876206189394,10.479999542236328,0.00055347615852952,130.6300048828125,1.8041261434555054,2.690000057220459,0.0,-24.700000762939453,False,0.0430000014603138,-99.0,True,False,2020,2020-08-29 02:10:38.245000+00:00,SB15670_15670_component_4273a,5.233333110809326,0.9203821420669556,0.00027777778450399637,0.00027777778450399637,0.0003388577897567302,0.0004381610488053411,0.0004381610488053411,5208737.5,5208737.5,False,1.0,1.0,21152927
3,3363670,SB10336_VAST_0918-06A_island_5046,SB10336_VAST_0918-06A_component_5046a,0.3784455955028534,138.08860778808594,2.7777778086601757e-06,-3.4262747764587402,2.7777778086601757e-06,1.5307477712631226,0.3784455955028534,1.5307477712631226,0.3784455955028534,25.76236915588379,0.0,11.812357902526855,0.0,66.01671600341797,0.0,25.76236915588379,11.812357902526855,66.01671600341797,False,976.1027221679688,0.0,False,False,1422,2019-10-29 18:40:19.293000+00:00,'image_SB10336_VAST_0918-06A_component_5046a_f_r...,4.04482889175415,1.0,0.00027777778450399637,0.00027777778450399637,0.0,0.00027779166703112423,0.00027779166703112423,12958704.0,12958704.0,True,1.0,1.0,21673913
4,3363670,SB11215_island_6860,SB11215_component_6860a,0.5135476589202881,138.08860778808594,2.7777778086601757e-06,-3.4262747764587402,2.7777778086601757e-06,-1.667264699935913,0.5135476589202881,-1.667264699935913,0.5135476589202881,18.84198570251465,0.0,11.163959503173828,0.0,-75.08185577392578,0.0,18.84198570251465,11.163959503173828,-75.08185577392578,False,543.3484497070312,0.0,False,False,1558,2020-01-11 17:29:11.326000+00:00,image_SB11215_component_6860a_f_run000007,-3.2465627193450928,1.0,0.00027777778450399637,0.00027777778450399637,0.0,0.00027779166703112423,0.00027779166703112423,12958704.0,12958704.0,True,1.0,1.0,21688501
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..